In [1]:
from pre_procesamiento import pre_process
from hashing import hash_signature_pool  
import pandas as pd
from collections import defaultdict
import pickle
import numpy as np
import itertools
import random
from ray.util.multiprocessing import Pool
import time
import psutil
start = time.time()

/Users/matif/.pyenv/versions/3.10.0/envs/tareas_proce/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# I. Pre-procesamiento de tweets:
Las siguientes transformaciones fueron aplicadas al texto que posteriormente se usó para computar las firmas de hash.
* Quitamos los retweets
* Removemos stopwords del español
* Quitamos los emoji
* Quitamos los tweets que contienen un enlace, ya que suelen ser poco interesantes al limitarse a compartir ese enlace sin mayor comentario
* Eliminamos los tweets que, tras aplicar todo el procesamiento anterior, tengan menos de 40 caracteres, ya que ralentizan la ejecución sin ser un mayor aporte a los resultados.

Cada chunk fue guardado en disco por separado con pickle y eliminado de memoria.

In [2]:
%%capture --no-stdout

chunksize = 2500000
shingles_dictionary = dict() # Diccionario que contendrá los set de shingles
for indice, chunk in enumerate(pd.read_csv("tweets_2022_abril_junio.csv", usecols=['screen_name', 'text'], chunksize=chunksize)):
        if indice == 0:
                prev_max_index = -1
        update_dict, prev_max_index = pre_process(chunk, indice, prev_max_index+1)
        shingles_dictionary.update(update_dict)
del chunk
del update_dict

# II. MinHash

Calculamos las firmas de hash usando el algoritmo [SuperMinHash](https://arxiv.org/abs/1706.05698). Paralelizamos con 4 nucleos para acortar tiempo. La cantidad de nucleos empleados responde a la cantidad de nucleos disponibles en la máquina al momento de ejecutar.

In [3]:
#%%capture --no-stdout
core_count = psutil.cpu_count() # Obtener nucleos disponibles
if core_count >= 6:
    partition_n = 4 # Si tienes más de 6, usas 4
else:
    partition_n = max(1, core_count//2) # Si tienes menos, usas la mitad 
lista_dict = list(shingles_dictionary.items())
largo = len(shingles_dictionary) // partition_n
dicts = [dict(lista_dict[:largo])]
for i in range(1,partition_n):
    if i != 3:
        dicts.append(dict(lista_dict[largo*i:largo*(i+1)]))
    else:
        dicts.append(dict(lista_dict[largo*i:]))
with Pool() as p:
    p.map(hash_signature_pool, [(dicc, 20, indice) for indice, dicc in enumerate(dicts)])

2023-06-05 16:33:32,477	INFO worker.py:1625 -- Started a local Ray instance.


Cargas las firmas de hash y consolidar en un único objeto

In [4]:
with open("fhs/file0.obj", 'rb') as file:
    FH = pickle.load(file)

for i in range(1, partition_n):
    with open(f"fhs/file{i}.obj", 'rb') as file:
        FH2 = pickle.load(file)
        FH = np.concatenate((FH, FH2), axis=1)
    del FH2
FH.shape

(20, 750862)

Volvemos a cargar los tweets pre-procesados en memoria.

In [5]:
with open('processed_tweets/resumen0.obj', 'rb') as file:
    resumen = pickle.load(file)
with open('processed_tweets/resumen1.obj', 'rb') as file:
    resumen = pd.concat([resumen, pickle.load(file)])

# III. LSH

Definimos parámetros para aplicar la técnica de banding

In [6]:
r = 10
b = 2
t = round((1/b)**(1/r), 2)
t

0.93

Nuestra elección de b y r fija el umbral en 0.93.

In [7]:
# Tomado de https://towardsdatascience.com/locality-sensitive-hashing-how-to-find-similar-items-in-a-large-set-with-precision-d907c52b05fc
n, d = FH.shape
hashbuckets = defaultdict(set)
bands = np.array_split(FH, b, axis=0)
for i,band in enumerate(bands):
    for j in range(d):
        # The last value must be made a string, to prevent accidental
        # key collisions of r+1 integers when we really only want
        # keys of r integers plus a band index
        band_id = tuple(list(band[:,j])+[str(i)])
        hashbuckets[band_id].add(j)
candidate_pairs = set()
for bucket in hashbuckets.values():
    if len(bucket) > 1:
        for pair in itertools.combinations(bucket, 2):
            candidate_pairs.add(pair)
print(f"Obtuvimos {len(candidate_pairs)} pares candidatos, con b = {b} y r = {r}")

Obtuvimos 193907 pares candidatos, con b = 2 y r = 10


Podamos los tweets idénticos entre sí

In [8]:
trimmed_cadidate_pairs = []
for pair in candidate_pairs:
        a, b = pair
        text_a = resumen.text[a]
        text_b = resumen.text[b]
        if (text_a != text_b):
                trimmed_cadidate_pairs.append((a, b))
print(f"Quedan {len(trimmed_cadidate_pairs)} pares candidatos")

Quedan 142280 pares candidatos


Mostrar 5 ejemplos de tweets semejantes

In [9]:
for pair in random.sample(trimmed_cadidate_pairs, 5):
    a, b = pair
    print(f"1. {resumen.screen_name[a]}: ",resumen.text[a])
    print(f"2. {resumen.screen_name[b]}: ",resumen.text[b])
    print("----------")

1. hernanm71450955:  @fernando_atria @christianpviera @letelier_raul @tatiurru @amaya_alvez @vichomrtnz @marian_barreaux @l_eslava Armonizar qué?
2. nicagandozurdo:  @fernando_atria @christianpviera @letelier_raul @tatiurru @amaya_alvez @vichomrtnz @marian_barreaux @l_eslava Armonizate la neurona
----------
1. cotecid:  @patriciapolitz @convencioncl @rkatrileo @MillaburAdolfo @ElisaLoncon @NoNeutrales No me gusta!!!
2. Antoniocl_35:  @patriciapolitz @convencioncl @rkatrileo @MillaburAdolfo @ElisaLoncon @NoNeutrales Ya y?
----------
1. Academica2021:  @OssandonLira @cgajardop @convencioncl @felipeharboe Cara de raja
2. fabio_fergo:  @OssandonLira @cgajardop @convencioncl @felipeharboe Te dejaron de wn
----------
1. jackyri18641013:  @RafaelTorrebl18 @tere_marinovic @rocicantuarias @Ktymontealegre 👍👍👍
2. roja_colorada:  @RafaelTorrebl18 @tere_marinovic @rocicantuarias @Ktymontealegre Minoría perdedora!
😅😂😁😄😆🤣😂🤣😆😄
----------
1. JCCONTACTOSUR:  @delosquesobran @danielstingo @PabloPincel
2.

Formatear como df

In [10]:
result = []
for pair in trimmed_cadidate_pairs:
    a, b = pair
    result.append([resumen.text[a], resumen.text[b], resumen.screen_name[a], resumen.screen_name[b]])

similar_tweets_df = pd.DataFrame(result, columns=['tweet_1', 'tweet_2', 'author_1', 'author_2'])

,tweet_1,tweet_2,author_1,author_2
0,@fernando_atria @christianpviera @letelier_rau...,@fernando_atria @christianpviera @letelier_rau...,LacombC,MariaPo78178726
1,Por eso #YoVotoRechazo #RechazoElMamarracho #R...,#RechazoGanaEl4deSeptiembre \n#RechazoElMamarr...,nanocarrasco42,mariellaandrea
2,@Parisi_oficial @CaneloCarola @fernando_atria ...,@Parisi_oficial @CaneloCarola @fernando_atria ...,rocuantluz,Dharmabitch1
3,@delosquesobran @danielstingo @PabloPincel,@delosquesobran @danielstingo Jajajajajajajaja...,JCCONTACTOSUR,JulioDo89187637
4,@fernando_atria @christianpviera @letelier_rau...,@fernando_atria @christianpviera @letelier_rau...,xuxumeitor,nakata1970
...,...,...,...,...
142275,@gdominguez_ @MEQChile @ElisaLoncon @Jaime_Bas...,@gdominguez_ @MEQChile @ElisaLoncon @Jaime_Bas...,JaquesSidoseux,TurtleBlind
142276,#RechazoCrece #RechazoTransversal #RechazoSalv...,#RechazoCrece #RechazoTransversal #RechazoSalv...,javalco16,javalco16
142277,@mcubillossigall @gabrielboric @izkia Shi shi,@mcubillossigall @gabrielboric Jajajajajaja,fherrerasoto,rodriguezdumbo
142278,@delosquesobran @danielstingo QUE MIEDO LES DA!!!,@delosquesobran @danielstingo JAJAJAJAJAJAJAJA...,Valeria08925319,PAVLOLP


# Autores Similares

In [11]:
count_dict = dict()

def def_value():
    return 0
for pair in trimmed_cadidate_pairs:
    a, b = pair
    user_a = resumen.screen_name[a]
    user_b = resumen.screen_name[b]
    if user_a != user_b:
        try:
            count_dict[user_a][user_b] += 1
        except KeyError:
            count_dict[user_a] = defaultdict(def_value) # No retorna key error
            count_dict[user_a][user_b] += 1
            
        try:
            count_dict[user_b][user_a] += 1
        except KeyError:
            count_dict[user_b] = defaultdict(def_value) # No retorna key error
            count_dict[user_b][user_a] += 1
            

Mostrar solo los autores que están por sobre un threshold de tweets similares

In [12]:
count_df = pd.DataFrame(count_dict).reset_index()
count_long = pd.melt(count_df, id_vars='index')
similar_authors = count_long[count_long.value >= 25].rename({'index': 'user_1', 'variable': 'user_2', 'value':'sim_tweet_count'}, axis = 1)
similar_authors = similar_authors.sort_values("sim_tweet_count", ascending=False)
similar_authors

,user_1,user_2,sim_tweet_count
796938,x1educalidad,osotroncoso,338.0
796944,malahierba84,osotroncoso,338.0
4509834,osotroncoso,malahierba84,338.0
807914,osotroncoso,x1educalidad,338.0
14798994,osotroncoso,zarate620,182.0
...,...,...,...
39416762,osotroncoso,JuanJoseNahuel,28.0
4509784,paolaespejo7,malahierba84,27.0
1842192,malahierba84,paolaespejo7,27.0
5369986,osotroncoso,imaureirac,26.0


Función para imprimir tweets de dos candidatos pares:

In [13]:
def show_tweets_from_candidates(user_1, user_2, amount):
    iterations = 0
    for pair in trimmed_cadidate_pairs:
        a, b = pair
        user_a = resumen.screen_name[a]
        user_b = resumen.screen_name[b]
        if ((user_a == user_1) and (user_b == user_2)) or ((user_b == user_1) and (user_a == user_2)):
            print("By user ", user_a,":\n",  f'"{resumen.text[a]}"\n')
            print("By user ", user_b, ":\n", f'"{resumen.text[b]}"')
            print("_______________________________________________________\n")
            if iterations == (amount-1):
                break
            iterations+=1

Veamos dos ejemplos de pares de tweets escritos por autores similares.

In [14]:
show_tweets_from_candidates("x1educalidad", "osotroncoso", 2)

By user  osotroncoso :
 "#Apruebo4deSeptiembre 
#AprueboPlebiscitoDeSalida #AprueboNuevaConstitucion #Apruebo #AprueboDeSalida"

By user  x1educalidad :
 "#AprueboDeSalida 
#Apruebo 
#Apruebo4deSeptiembre   
#AprueboNuevaConstitución 
#AprueboPlebiscitoDeSalida"
_______________________________________________________

By user  osotroncoso :
 "#Apruebo4deSeptiembre #AprueboPlebicitoDeSalida #AprueboNuevaConstitucion
#AprueboDeSalida #Apruebo"

By user  x1educalidad :
 "#AprueboPlebiscitoDeSalida 
#Apruebo4deSeptiembre 
#apruebo
#AprueboNuevaConstitucion"
_______________________________________________________



A continuación podemos examinar un cierto número de tweets para un n dado de autores similares.

In [15]:
def shown_n_similar_authors_examples(n_examples_per_pair, n_author_pairs):
    iteration = 0
    for index, row in similar_authors.iterrows():
        user_1 = row.user_1
        user_2 = row.user_2
        show_tweets_from_candidates(user_1, user_2, n_examples_per_pair) # Sacar 3 ejemplos por cada par
        if iteration == (n_author_pairs-1):
            break
        iteration += 1
shown_n_similar_authors_examples(3, 2)
# Se mostraran 3 ejemplos para 2 pares de autores similares: 

By user  xuxumeitor :
 "@delosquesobran @danielstingo Cállate CTM!!!"

By user  JCCONTACTOSUR :
 "@delosquesobran @danielstingo @PabloPincel"
_______________________________________________________

By user  xuxumeitor :
 "@delosquesobran @danielstingo Cállate CTM!!!"

By user  JCCONTACTOSUR :
 "@delosquesobran @danielstingo @PabloPincel"
_______________________________________________________

By user  xuxumeitor :
 "@delosquesobran @danielstingo Cállate CTM!!!"

By user  JCCONTACTOSUR :
 "@delosquesobran @danielstingo @PabloPincel"
_______________________________________________________

By user  clavijerosexto :
 "@delosquesobran @danielstingo Wn penca jajaja"

By user  JCCONTACTOSUR :
 "@delosquesobran @danielstingo @PabloPincel"
_______________________________________________________

By user  JCCONTACTOSUR :
 "@delosquesobran @danielstingo @PabloPincel"

By user  clavijerosexto :
 "@delosquesobran @danielstingo Wn penca jajaja"
____________________________________________________

In [16]:
tiempo_total = time.time() - start
tiempo_total / 60

5.625787381331126